# CS202: Compiler Construction

## In-class Exercises, Week of 02/06/2023

----

Notes:

            Size      Speed
Registers   ~15       incredibly fast
Memory      lots      slow

So avoiding memory and using registers as much as possible is the biggest optimization
there is. (The other is static type checking in a month)

# Register allocation

*Register allocation* is the process of assigning register locations to program variables.

## Question 1

Why is register allocation important?

It makes the programs a lot faster. Registers are much faster than memory. Probably the biggest optimization
you can do.

## Question 2

Consider the program:

```
x = 5
y = x + 6
y + 7
```

Here is the equivalent in x86 assembly, using the stack to hold variable values.

```
...
start:
  movq $5, -8(%rbp)
  movq -8(%rbp), %rax
  movq %rax, -16(%rbp)
  addq $6, -16(%rbp)
  movq -16(%rbp), %rax
  movq %rax, -24(%rbp)
  addq $7, -24(%rbp)
  movq -24(%rbp), %rax
  jmp conclusion
...
```

Convert this program into one that only uses registers.

lets assign each stack location with a register instead, then find and replace.

textbook shows registers and calling convention
rax: tmp location (so don't want to use that)

lets use r8 through r11

-8 -> r8
-16 -> r9
-24 -> r10
```
...
start:
  movq $5, -8(%rbp)
  movq -8(%rbp), %rax
  movq %rax, -16(%rbp)
  addq $6, -16(%rbp)
  movq -16(%rbp), %rax
  movq %rax, -24(%rbp)
  addq $7, -24(%rbp)
  movq -24(%rbp), %rax
  jmp conclusion
...
```

becomes:

```
...
start:
  movq $5, %r8
  movq %r8, %rax
  movq %rax, %r9
  addq $6, %r9
  movq %r9, %rax
  movq %rax, %r10
  addq $7, %r10
  movq %r10, %rax
  jmp conclusion
...
```

## Question 3

How many registers are *actually needed* to represent this program?

Could have used r8 for all of these. All the vairbale can share one location without changing the meaning of the program.

Could have used %rax!

## Question 4

Why is *optimal* register allocation hard?

This easy to do by hand but coming up with a general form for the compiler to do.
Every variable getting a register is a waste.

Optimal register allocation is actually NP-Complete! (not easy)

We need an approximately optimal algorithm that runs fast enough for practical use.
(we'll be using a quadratic time one)



## Note

Our solution to register allocation will involve three parts:

1. [*Liveness analysis*](https://en.wikipedia.org/wiki/Live_variable_analysis), to understand when a variable's value is no longer needed
2. Building an *interference graph* with edges between variables that cannot share a register
3. Allocating registers to variables by *coloring the interference graph* ([graph coloring](https://en.wikipedia.org/wiki/Graph_coloring) is NP-hard)

Interference graph will be a graph where variables are nodes in a graph
an edge between them mean they interfere. i.e. storing them in the same location
will break the program.

GOAL:

1) Try to minimize number of locations will maximize the register usage.
    - Liveness Analysis

2) We will do this by maximizing sharing of locations

3) Can x, y share a location? => Are x and y live at same time?
    - Interference Graph
    - Graph Coloring
   

----

# Liveness Analysis

The first part of the allocate-registers pass performs a *liveness analysis* on the program.

This is an example of a program analysis (a program that looks at other programs)

What does it mean for a variable to be live?
 - x is live is we will need its value later
     - Ex: 
           x = 5 + 6 (x live)
           y = x + 7 (y live)
           z = y + 8 (nothing live)
 - So at each line we make a set of live variables
 
The algorithm to do this is O(n^2) if we start from top and scan for when things are used.

This becomes O(n) if we start from the bottom:
 - Start at bottom: Empty set of variables
 - Next line up: add Cars read from, remove vars written to
 
So why in x86 instead of python?
 - variables are set
 - control flow (not an issue in this assignment, but will be later)

## Question 5

Here is the pseudo-x86 equivalent of the program from earlier, written in x86 assembly syntax:

```
start:
 movq $5, x
 movq x, y
 addq $6, y
 movq y, r
 addq $7, r
 movq r, %rax
```

In the table below, annotate each instruction with its *live-after set*.

Use the following rule to determine the live-after set for instruction $k$:

\begin{align}
L_{after}(k) = (L_{after}(k+1) − W (k+1)) \cup R(k+1)
\end{align}

where $W$ means the variables written by the instruction, and $R$ means the variables read by an instruction. See section 3.2 in the textbook for more.

## Question 6

Describe the liveness analysis in the compiler.

Big Idea:
   - Process instructions in reverese order
   - STart with empty live-after set
   - Produce the next live-after set using the previous one based on previously stated rule
    
Structure:
   - Wre can use two functions:
       - `ul_instrs`: given instructions k+1 and the live-after set for instructions k+1, produce the live-after set for instructions k (use rule)
       - `ul_block`: 
           - Start with empty list live-after sets
           - start with empty current live-after set
           - loop over ionstructions in reverese order
               - call `ul_instr` to get the next live after-set
               - add it to the list
           - at the end, reverese the liost of live-after sets and return it

----

# Interference Graph

The second part of the allocate-registers pass will build an *interference graph* using the live-after sets computed earlier. The interference graph is an undirected graph with a node for each program variable, and an edge between each pair of variables which may *not* share a home.

## Question 7

Using the instructions and live-after sets from question 5, draw the interference graph.

Use the following principle: *the writes performed by an instruction must not overwrite something in a live location*.

Interference Graph has no edges, all the variables can go to same location

## Question 8

Using the instructions and live-after sets below, draw the interference graph.

| Instruction    | Live-After Set |
| :------------- | :------------- |
| `start:`       | {} |
| `movq $1, a`   | {a}|
| `movq $2, b`   | {a, b} |
| `movq $3, c`   | {a, b, c} |
| `movq a, t`    | {b, c, t} |
| `addq b, t`    | {c, t} |
| `movq t, r`    | {c, r} |
| `addq c, r`    | {r} |
| `movq r, %rax` | {}  |
  _____
 /     \
a---b---c
     \ / \
  r   t  /
   \____/

## Question 9

What variables in the above graph can share a register? What variables can't?

YOUR ANSWER HERE